In [1]:
ljy_fp = r"D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmark.asm"
GuGuJi_fp = r"D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmark.asm"
GuGuJi_spec_fp = r"D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmark.asm"
GuGuJi_oldfp = r"/benchmarks/old_benchmark/spec_example/spec2006_470.lbm/lbm_part.asm"
why_fp = r"D:\workspace\Gitdocuments\hw-memory\benchmarks\final_benchmark\spec_benchmark.asm"
fp = ljy_fp
guguji_segfp = r"D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmarkD.asm"
seg_fp = r'D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmarkD.asm'
why_fp = r'D:\workspace\Gitdocuments\hw-memory\benchmarks\final_benchmark\spec_benchmarkD.asm'


In [2]:
'''读指令, 指令对象化'''
from newCFG.cfg import proc_identify
from newCFG.isa import Instruction, AddrMode
from typing import Tuple
from newCFG.read_asm import AsmFileReader, StatementType
from cache_analysis.read_segment import segmentReader

reader =AsmFileReader(fp)
segreader = segmentReader(seg_fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

        

for i in temp_stat:
     if i.is_ls:
        print(i.tokens,i.ls_reg_target,i.ls_reg_target_list,i.ls_addr_offset)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
print()

('400624', 'a9bf7bfd', 'stp', None, 'x29, x30, [sp, #-16]!') None [] 0
('400630', 'a8c17bfd', 'ldp', None, 'x29, x30, [sp], #16') None [] 0
('400640', 'a9bf7bf0', 'stp', None, 'x16, x30, [sp, #-16]!') None [] 0
('400648', 'f947fe11', 'ldr', None, 'x17, [x16, #4088]') x16 [] 4088
('400664', 'f9400211', 'ldr', None, 'x17, [x16]') x16 [] 0
('400674', 'f9400611', 'ldr', None, 'x17, [x16, #8]') x16 [] 8
('400684', 'f9400a11', 'ldr', None, 'x17, [x16, #16]') x16 [] 16
('400694', 'f9400e11', 'ldr', None, 'x17, [x16, #24]') x16 [] 24
('4006a4', 'f9401211', 'ldr', None, 'x17, [x16, #32]') x16 [] 32
('4006b4', 'f9401611', 'ldr', None, 'x17, [x16, #40]') x16 [] 40
('4006c4', 'f9401a11', 'ldr', None, 'x17, [x16, #48]') x16 [] 48
('4006d4', 'f9401e11', 'ldr', None, 'x17, [x16, #56]') x16 [] 56
('4006e4', 'f9402211', 'ldr', None, 'x17, [x16, #64]') x16 [] 64
('4006f4', 'f9402611', 'ldr', None, 'x17, [x16, #72]') x16 [] 72
('400710', 'f94003e1', 'ldr', None, 'x1, [sp]') sp [] 0
('400748', 'f947ec00',

In [3]:
'''分proc'''
from graphviz import Digraph
from newCFG.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output\\procedures.svg'

In [4]:
'''处理call_graph'''
from newCFG.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output\\call_graph.svg'

In [5]:
'''生成TCFG和loop信息'''
from newCFG.cfg import TCfg


tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()
tcfg.add_loop_bound(r'D:\workspace\hw-memory\benchmarks\loop_bound.txt')
# tcfg.add_loop_bound(r'D:\workspace\Gitdocuments\hw-memory\benchmarks\loop_bound.txt')
for l in tcfg.loops:
    print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    l.back_edge.is_backEdge = True
for n in tcfg.all_nodes:
    temp = n.inside_loop.name if n.inside_loop is not None else 'None'
    print("{} {}".format(n.name, temp))



#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




l1 (in l0 children []): n76->n77 ['n77', 'n94', 'n76', 'n75']
l5 (in l4 children []): n61->n62 ['n61', 'n62']
l4 (in l2 children []): n63->n64 ['n60', 'n61', 'n62', 'n63', 'n64']
l10 (in l6 children []): n53->n54 ['n53', 'n54']
l16 (in l11 children []): n43->n44 ['n41', 'n40', 'n42', 'n43', 'n44']
l23 (in l17 children []): n33->n34 ['n89', 'n31', 'n32', 'n30', 'n33', 'n34']
l31 (in l24 children []): n23->n24 ['n88', 'n24', 'n23', 'n21', 'n22', 'n20']
l0 (in None children []): n78->n79 ['n76', 'n77', 'n78', 'n79', 'n94', 'n74', 'n75']
l2 (in None children []): n65->n66 ['n66', 'n60', 'n61', 'n62', 'n59', 'n63', 'n64', 'n65']
l6 (in None children []): n55->n56 ['n52', 'n53', 'n54', 'n55', 'n56']
l11 (in None children []): n45->n46 ['n46', 'n41', 'n40', 'n42', 'n39', 'n43', 'n44', 'n45']
l17 (in None children []): n35->n36 ['n36', 'n89', 'n31', 'n32', 'n30', 'n29', 'n33', 'n34', 'n35']
l24 (in None children []): n25->n26 ['n23', 'n88', 'n24', 'n25', 'n26', 'n19', 'n20', 'n22', 'n21']
n0 N

In [6]:
from rwcond_out import loadstore_Obj
lds_obj = loadstore_Obj(segreader,tcfg)
lsproc = lds_obj.lsproc
ls_loop_info = lds_obj.loop_info

for k,v in ls_loop_info.items():
    print(k,v)

l1 {}
l5 {1065: -1, 1066: -1, 1067: -1, 1068: -1, 1069: -1, 1070: -1, 1071: -1, 1072: -1, 1073: -1, 1074: -1, 1075: -1}
l4 {1065: -1, 1066: -1, 1067: -1, 1068: -1, 1069: -1, 1070: -1, 1071: -1, 1072: -1, 1073: -1, 1074: -1, 1075: -1}
l10 {1056: -1, 1057: -1, 1058: -1, 1059: -1, 1060: -1, 1061: -1, 1062: -1, 1063: -1, 1064: -1, 1065: -1}
l16 {}
l23 {1056: 2.1625, 1057: 2.1625, 1058: 2.1625, 1059: 2.1625, 1060: 2.1625, 1061: 2.1625, 1062: 2.1625, 1063: 2.1625, 1064: 2.1625, 1065: 2.1625}
l31 {1056: 1.8375, 1057: 1.8375, 1058: 1.8375, 1059: 1.8375, 1060: 1.8375, 1061: 1.8375, 1062: 1.8375, 1063: 1.8375, 1064: 1.8375, 1065: 1.8375}
l0 {}
l2 {1065: -1, 1066: -1, 1067: -1, 1068: -1, 1069: -1, 1070: -1, 1071: -1, 1072: -1, 1073: -1, 1074: -1, 1075: -1}
l6 {1056: -1, 1057: -1, 1058: -1, 1059: -1, 1060: -1, 1061: -1, 1062: -1, 1063: -1, 1064: -1, 1065: -1}
l11 {}
l17 {1056: 2.1625, 1057: 2.1625, 1058: 2.1625, 1059: 2.1625, 1060: 2.1625, 1061: 2.1625, 1062: 2.1625, 1063: 2.1625, 1064: 2.1625, 10

In [7]:
# '''设置node和edge上的的value'''
# import random
# for l in tcfg.loops:
#     l.back_edge.is_backEdge = True
# for n in tcfg_nodes:
#     n.set_rw_data()

# temp_bool = True

# while temp_bool:
#     no_end = False

#     for n in tcfg_nodes:
#         n.set_rw_value()
#         if not n.no_out:
#             no_end = True
#         temp_bool = no_end

# for e in tcfg_edges:
#     if e.is_backEdge:
#         e.loop_value = random.randint(0,10)
#         e.edge_value = 2

# #for e in tcfg_edges:
# #    print(e.src.name,e.dst.name,e.edge_value)

In [8]:
# '''ls分析和读写特征'''

# from newCFG.cfg import TCfg
# from newCFG.loadstore import LSProc
# from newCFG.rw_condition import RWProc, RWType

# lsproc = LSProc(tcfg_nodes)

# lds_table = lsproc.ls_table

# for i in lds_table:
#     i.final_addr
#     i.local_offset

# #for i in lds_table:
# #    if i.target_num == 1:
#  #       pass
#         #print(i.ins.tokens,i.reg_target,i.addr_offset) 
#  #   else:
#         #print(i.ins.tokens,i.reg_target_list[0],i.reg_target_list[1],i.addr_offset,i.final_addr)

# rwproc = RWProc(lds_table)

# Global_Tolerant_value = 0
# Global_Intolerant_value = 0

# for rwu in rwproc.rw_table:
#     #print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)
#     if rwu.is_torrent == RWType.Global_Tolerant:
#         Global_Tolerant_value += rwu.find_cycle
#         print(rwu.ins.tokens,rwu.ins.final_addr,rwu.node.name)
# print()
# for rwu in rwproc.rw_table:
#     #print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)
#     if rwu.is_torrent == RWType.Global_Intolerant:
#         Global_Intolerant_value += rwu.find_cycle
#         print(rwu.ins.tokens,rwu.ins.final_addr,rwu.node.name)

# print("全局的容错路径为：",Global_Tolerant_value)
# print("全局的非容错路径为：",Global_Intolerant_value)   

In [9]:
# '''cache分析'''

# #打印给cache分许需要的数据
# for i in lds_table:
#     print("指令",i.ins.tokens,"是否是SP",i.is_sp,"指令地址",i.ins.addr.val(),"访存地址",i.final_addr,"数据宽度",i.ins.ls_data_width,"是否是加入一个数组",i.ins.is_data_group)


In [10]:
# from newCFG.heat_analysis import loop_heat
# test=loop_heat(tcfg,lsproc,r'C:\Users\13377\Desktop\hw-memory-master (5)\hw-memory-master\benchmarks\final_benchmark\spec_benchD.asm')
# test.do_it()


In [11]:

# TODO.
from cache_analysis.cache_risk_level import CacheRisk

cache_test = CacheRisk(tcfg, lsproc, r'D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmarkD.asm',
                       r'D:\workspace\hw-memory\cache_analysis\new_cache\input\cache_information.in')
cache_test.test()


[('n0', 'n12'), ('n17', 'n1'), ('n1', 'n4'), ('n1', 'n2'), ('n2', 'n18'), ('n27', 'n3'), ('n3', 'n28'), ('n37', 'n4'), ('n4', 'n38'), ('n50', 'n5'), ('n5', 'n9'), ('n5', 'n6'), ('n6', 'n51'), ('n57', 'n7'), ('n7', 'n58'), ('n67', 'n8'), ('n8', 'n9'), ('n9', 'n68'), ('n70', 'n10'), ('n10', 'n71'), ('n82', 'n11'), ('n12', 'n83'), ('n83', 'n13'), ('n13', 'n84'), ('n84', 'n14'), ('n14', 'n85'), ('n85', 'n15'), ('n15', 'n86'), ('n86', 'n16'), ('n16', 'n87'), ('n87', 'n17'), ('n18', 'n26'), ('n19', 'n24'), ('n20', 'n88'), ('n88', 'n21'), ('n21', 'n23'), ('n21', 'n22'), ('n22', 'n23'), ('n23', 'n24'), ('n24', 'n20'), ('n24', 'n25'), ('n25', 'n26'), ('n26', 'n19'), ('n26', 'n27'), ('n28', 'n36'), ('n29', 'n34'), ('n30', 'n89'), ('n89', 'n31'), ('n31', 'n33'), ('n31', 'n32'), ('n32', 'n33'), ('n33', 'n34'), ('n34', 'n30'), ('n34', 'n35'), ('n35', 'n36'), ('n36', 'n29'), ('n36', 'n37'), ('n38', 'n46'), ('n39', 'n44'), ('n40', 'n42'), ('n40', 'n41'), ('n41', 'n43'), ('n42', 'n43'), ('n43', 'n44')

[['l1', 1025, -1],
 ['l1', 1056, 1.1333120002133312],
 ['l1', 1057, 1.0],
 ['l1', 1058, 1.0],
 ['l1', 1059, 2.5550025551580866],
 ['l1', 1060, 1.0],
 ['l1', 1061, 1.0],
 ['l1', 1062, 1.0],
 ['l1', 1063, 2.5550025551580866],
 ['l1', 1064, 1.0],
 ['l1', 1065, 1.0],
 ['l1', 1024, 39999.0],
 ['l5', 1024, 27.666647555568296],
 ['l5', 1065, 0.23437472998052783],
 ['l5', 1066, 0.999998000002],
 ['l5', 1067, 2.5555492345791384],
 ['l5', 1068, 0.9844941856878496],
 ['l5', 1069, 0.9844941856878496],
 ['l5', 1070, 0.999998000002],
 ['l5', 1071, 2.5555492345791384],
 ['l5', 1072, 0.9844941856878496],
 ['l5', 1073, 0.9844941856878496],
 ['l5', 1074, 0.999998000002],
 ['l5', 1075, 0.999998000002],
 ['l5', 1056, 1.0],
 ['l5', 1025, 11.999987000013],
 ['l4', 1024, 21.498875056247186],
 ['l4', 1065, 0.2343479986375298],
 ['l4', 1066, 0.9998000199980002],
 ['l4', 1067, 2.5549235691432632],
 ['l4', 1068, 0.9843023084566934],
 ['l4', 1069, 0.9843023084566934],
 ['l4', 1070, 0.9998000199980002],
 ['l4', 10

In [12]:

# from newCFG.rw_condition import RWProc, RWType
# from newCFG.rw_condition_out import RWOut_Proc



# rwout = RWOut_Proc(tcfg_nodes,segreader,rwproc.rw_table,tcfg.loops)




